# Experiments with Fully Connected Dense Networks

* The Experiments are loggged in [Weights and Biases platform](https://wandb.ai/amitagni/candlestick-simple?workspace=user-amitagni)
* All the models were overfitting Training data
* Unsucessfully tried to fix using 
    + Batch Size reduction
    + L2 regularisation
    + Dropout regularisation

In [1]:
import tensorflow as tf
from tensorflow import keras
import time
import wandb
from wandb.keras import WandbCallback


from functions_dataCreation import *
from functions_modelArchitectures import *
from class_LRFinder import *

set_x,set_y = readXYfromDisk(noOfFiles=27,folder="../data/Train") #Training data
test_set_x,test_set_y = readXYfromDisk(noOfFiles=2,folder='../data/Test') 

set_y[set_y == 1] = 0
set_y[set_y == 2] = 1

test_set_y[test_set_y == 1] = 0
test_set_y[test_set_y == 2] = 1

IMG_SIZE = 128


X Shape :  (114364, 128, 128, 3) Memory size is : 5360.8125 Mb Y Shape:  (114364,)
Values, counts, Avg Performance :  [0 1 2] [59258 26852 28254] [0.51815257 0.23479417 0.24705327]
X Shape :  (10060, 128, 128, 3) Memory size is : 471.5625 Mb Y Shape:  (10060,)
Values, counts, Avg Performance :  [0 1 2] [4988 2453 2619] [0.49582505 0.24383698 0.26033797]


## Create model structure

In [ ]:
IMG_SIZE = 128
modelBase = {}
modelBase['inputShape'] = (IMG_SIZE,IMG_SIZE,3)
modelBase['units'] = 128
modelBase['activation'] = 'relu'

# Large
modelLarge =  modelBase.copy()
modelLarge['name'] = 'Large'
modelLarge['layers'] = 24

modelLarge_Dropout = modelLarge.copy()
modelLarge_Dropout['name'] = 'Large_Dropout'


METRICS = [
    #   keras.metrics.TruePositives(name='tp'),
    #   keras.metrics.FalsePositives(name='fp'),
    #   keras.metrics.TrueNegatives(name='tn'),
    #   keras.metrics.FalseNegatives(name='fn'), 
    #   keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]



## Batch Size 256

In [4]:
for lrnRate in [1e-7,5e-7,1e-6]:
    for doRate in [0.2,0.4,0.6]:

        start_time = time.time()
        experName = "L24_U128_BS_256_RMSPropLR_" + str(lrnRate) + "_DropoutRate_" + str(doRate)
        print(experName)
        wandb.init(project="candlestick-simple", name = experName )

        modelLarge_Dropout['dropout'] = doRate
        model = createFullyConnected(modelLarge_Dropout)
        model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=lrnRate)
                        ,loss=tf.keras.losses.binary_crossentropy
                        ,metrics=[METRICS])

        history = model.fit(set_x, set_y
                            ,batch_size = 256
                            ,epochs=1000
                            ,verbose=0
                            ,validation_data=(test_set_x,test_set_y)
                            ,validation_freq = 50
                            ,callbacks=[WandbCallback()]
                            )
        print("Elapsed time ", round((time.time() - start_time)/60,2))



L24_U128_BS_256_RMSPropLR_2e-05_DropoutRate_0.1
Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: amitagni (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
Elapsed time  189.54
L24_U128_BS_256_RMSPropLR_2e-05_DropoutRate_0.3


epoch,999
loss,0.27659
precision,0.81624
recall,0.70525
auc,0.93044
_step,999
_runtime,11371
_timestamp,1604232654
val_loss,7.92099
val_precision,0.26458
val_recall,0.29095


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█████▇▇▇▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▅▁▁▁▁▁▁▁▁
precision,▁▁▁▁▆▆▆▆▇▇▇▇▇▇▇▇▇███████████████████████
recall,▁▁▁▁▁▁▂▃▃▄▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇██████████████
auc,▁▁▂▂▂▃▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇██████████████████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,▁▁▁▁▁▁▁▁▂▂▂▂▂▃▂▃▂▂▄█
val_precision,▁█▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
val_recall,▁▁▂▃▂▆▅▄▄▄▆▅▇▆▅▇▄▃▅█


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
Elapsed time  189.47
L24_U128_BS_256_RMSPropLR_2e-05_DropoutRate_0.5


...

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁
precision,▁▁▁▁▆▇▇▇▇▇▇▇████████████████████████████
recall,▁▁▁▁▁▂▂▃▃▄▄▅▅▆▆▆▆▆▆▆▆▇▆▇▇▆▇▇▇▇▇▇▇▇▇█████
auc,▁▁▂▂▃▃▄▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇▆▇▇▇▇▇▇▇▇▇████████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,▁▁▁▁▁▂▂▃▃▄█▄▆▆▇▃▆▃▂▄
val_precision,▁██▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
val_recall,▁▁▂▂▃▃▃▃▄▃▄▃▄▄▂▃█▄▃▅


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
Elapsed time  191.41
L24_U128_BS_256_RMSPropLR_3e-05_DropoutRate_0.1


epoch,999
loss,0.41495
precision,0.70732
recall,0.4352
auc,0.8353
_step,999
_runtime,11459
_timestamp,1604255507
val_loss,0.84575
val_precision,0.26416
val_recall,0.15311


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,████▇▇▇▆▆▅▅▅▄▄▄▄▄▆▃▄▃▃▃▄▃▃▄▃▃▂▂▂▂▃▂▁▁▁▁▃
precision,▁▁▁▁▁▆▇▇▇▇▇▇▇███████████████████████████
recall,▁▁▁▁▁▂▂▂▃▃▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇██████
auc,▁▁▂▂▃▃▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,▁▁▁▁▁▁▂▂▁▂▃▂█▂▃▂▂▂▂▂
val_precision,▁▁▆▇▇▇▇█▇▇▇▇▇█▇█▇█▇▇
val_recall,▁▁▂▂▄▂▂▃▃▆█▄▄▂▃▃▅▃▅▄


KeyboardInterrupt: 

## Batch Size 128

In [3]:
IMG_SIZE = 128
modelBase = {}
modelBase['inputShape'] = (IMG_SIZE,IMG_SIZE,3)
modelBase['units'] = 128
modelBase['activation'] = 'relu'

# Medium
modelMedium =  modelBase.copy()
modelMedium['name'] = 'Medium'
modelMedium['layers'] = 12

modelMedium_Dropout = modelMedium.copy()
modelMedium_Dropout['name'] = 'Medium_Dropout'
modelMedium_Dropout['dropout'] = 0.5



METRICS = [
    #   keras.metrics.TruePositives(name='tp'),
    #   keras.metrics.FalsePositives(name='fp'),
    #   keras.metrics.TrueNegatives(name='tn'),
    #   keras.metrics.FalseNegatives(name='fn'), 
    #   keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]



for lrnRate in [1e-6,5e-6,1e-5]:
    for doRate in [0.2,0.4,0.6]:

        start_time = time.time()
        experName = "L12_U128_BS_64_RMSPropLR_" + str(lrnRate) + "_DropoutRate_" + str(doRate)
        print(experName)
        wandb.init(project="candlestick-simple", name = experName )

        modelMedium_Dropout['dropout'] = doRate
        model = createFullyConnected(modelMedium_Dropout)
        model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=lrnRate)
                        ,loss=tf.keras.losses.binary_crossentropy
                        ,metrics=[METRICS])

        history = model.fit(set_x, set_y
                            ,batch_size = 64
                            ,epochs=1000
                            ,verbose=0
                            ,validation_data=(test_set_x,test_set_y)
                            ,validation_freq = 50
                            ,callbacks=[WandbCallback()]
                            )
        print("Elapsed time ", round((time.time() - start_time)/60,2))



L12_U128_BS_64_RMSPropLR_1e-06_DropoutRate_0.2
Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: amitagni (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
Elapsed time  359.82
L12_U128_BS_64_RMSPropLR_1e-06_DropoutRate_0.4


epoch,999
loss,0.23627
precision,0.84186
recall,0.76796
auc,0.94804
_step,999
_runtime,21589
_timestamp,1604308036
val_loss,1.14139
val_precision,0.26008
val_recall,0.13058


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██████▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
precision,▁▁▁▁▇▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
recall,▁▁▁▁▁▁▁▂▂▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇███████
auc,▁▁▂▂▂▃▃▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██████████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,▁▁▁▁▂▂▂▃▃▅▄▅▅▅▆▆██▇█
val_precision,▁▁██▇▇▇▇█▇▇▇▇█▇▇▇█▇▇
val_recall,▁▁▁▂▂▃▃▃▃█▄▅▆▃▄▃▂▂▄▃


KeyboardInterrupt: 